# Building Graph-based Applications

This notebook shows a way to use `ipyradiant` widgets to build a simple tab-based
application.

In [ ]:
import traitlets as T

import ipywidgets as W
from ipyradiant import CytoscapeVisualizer, FileManager, PathLoader, QueryWidget
from rdflib import BNode, Graph

## The App Class

This class-based widget captures both the overall layout, as well its individual
components. Values are
[linked](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html#Linking-Widgets)
to create new behaviors.

In [ ]:
class RadiantTabs(W.Tab):
    graph = T.Instance(Graph, allow_none=True)
    load_widget = T.Instance(FileManager)
    query_widget = T.Instance(QueryWidget)
    vis_widget = T.Instance(CytoscapeVisualizer)
    log = W.Output()

    def __init__(self, graph: Graph = None, *args, **kwargs):
        if "layout" not in kwargs:
            kwargs["layout"] = dict(min_height="800px")
        super().__init__(*args, **kwargs)
        T.link((self.load_widget, "graph"), (self, "graph"))
        T.link((self, "graph"), (self.query_widget, "graph"))
        T.link((self, "graph"), (self.vis_widget, "graph"))

        if graph:
            self.graph = graph
            self.graph_id = graph.identifier

        self.children = [self.load_widget, self.query_widget, self.vis_widget]
        self.set_title(0, "RDF Loader")
        self.set_title(1, "Query Panel")
        self.set_title(2, "Vis Panel")

    @T.default("graph")
    def make_default_graph(self):
        return Graph()

    @T.default("load_widget")
    def make_default_load_widget(self):
        return FileManager(loader=PathLoader(path="data"))

    @T.default("query_widget")
    def make_default_query_widget(self):
        return QueryWidget()

    @T.default("vis_widget")
    def make_vis_widget(self):
        return CytoscapeVisualizer()

## Showing the App

In [ ]:
tabs = RadiantTabs()
W.VBox([tabs, tabs.query_widget.log])